In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cond_rnn

from sklearn.preprocessing import MinMaxScaler
from solver import processing
from solver import lstm

Using TensorFlow backend.


In [2]:
### FEATURE PARAMETERS
# prediction target
feature_predict = 'P_avg'
# prediction inputs from turbine data
features_train = ['P_avg', 'Ba_avg', 'Wa_avg', 'Wx', 'Wy']
# engineered prediction inputs 
features_cond = ['Day sin', 'Day cos', 'Year sin', 'Year cos']

### TRAIN/VAL/TEST SPLIT
train_years = [2013, 2014, 2015, 2016]
validation_years = [2017]
test_years = [2017]

### TRAINING PARAMETERS:
BATCH_SIZE = 512
N_EPOCHS = 100
N_NEURONS = 64

### FEATURE ENGINEERING PARAMETERS
MA_CONSTANT = 3 # moving average smoothing parameter
N_OUT = 12 # forecast horizon
N_PAST = 48 # number of autoregression samples

In [3]:
# define datasets
TURBINE_ID = 'R80711'
DATA_DIR = os.path.join('../datasets/after_imputation', 'turbine_{}.csv'.format(TURBINE_ID))

# read datasets
dataset = processing.read_dataset(DATA_DIR)

In [4]:
# define masks for training/validation and testing (will be used later)
train_idx = dataset[dataset['Date_time'].dt.year.isin(train_years)].index
valid_idx = dataset[dataset['Date_time'].dt.year.isin(validation_years)].index
test_idx = dataset[dataset['Date_time'].dt.year.isin(test_years)].index

In [5]:
# some stats:
print("Number of duplicates: \t\t {}".format(len(dataset.index[dataset.index.duplicated()].unique())))
print("Number of rows with nan: \t {}".format(np.count_nonzero(dataset.isnull())))

# perform smoothing
if feature_predict in features_train:
    dataset = processing.smooth(dataset, cols_to_smooth=features_train, ma_constant=MA_CONSTANT)
else:
    dataset = processing.smooth(dataset, cols_to_smooth=features_train+[feature_predict], ma_constant=MA_CONSTANT)

Number of duplicates: 		 0
Number of rows with nan: 	 0


In [6]:
# define dates for plotting
test_dates = dataset.loc[dataset['Date_time'].dt.year.isin(test_years), 'Date_time'].values

# split to training/validation/testing sets based on indices
dataset_train = dataset[dataset.index.isin(train_idx)].copy()
dataset_valid = dataset[dataset.index.isin(valid_idx)].copy()
dataset_test = dataset[dataset.index.isin(test_idx)].copy()

# define target mask for features
target_idx = np.where(dataset_train.columns == feature_predict)[0][0]
target_mask = np.zeros((dataset_train.shape[1])).astype(bool)
target_mask[target_idx] = True
# define input mask for autoregression features
input_idx = [np.where(dataset_train.columns == feat_col)[0][0] for feat_col in features_train+features_cond]
input_mask = np.zeros((dataset_train.shape[1])).astype(bool)
input_mask[input_idx] = True

# apply masks
y_train = dataset_train.iloc[:, target_mask]
y_valid = dataset_valid.iloc[:, target_mask]
y_test = dataset_test.iloc[:, target_mask]
X_train = dataset_train.iloc[:, input_mask]
X_valid = dataset_valid.iloc[:, input_mask]
X_test = dataset_test.iloc[:, input_mask]

# define mask for conditional features
cond_idx = [np.where(X_train.columns == feat_col)[0][0] for feat_col in features_cond]
cond_mask = np.zeros((X_train.shape[1])).astype(bool)
cond_mask[cond_idx] = True

# Define scaler and fit only on training data
scaler_output = MinMaxScaler()
y_train = scaler_output.fit_transform(y_train)
y_valid = scaler_output.transform(y_valid)
y_test = scaler_output.transform(y_test)
# Define scaler and fit only on training data
scaler_input = MinMaxScaler()
X_train = scaler_input.fit_transform(X_train)
X_valid = scaler_input.transform(X_valid)
X_test = scaler_input.transform(X_test)

# Make small tests
assert X_train.shape[0] == y_train.shape[0]
assert X_valid.shape[0] == y_valid.shape[0]
assert X_test.shape[0] == y_test.shape[0]

In [7]:
# Split X variables to autoregression and conditional variables
X_train_ar = X_train[:, ~cond_mask]
X_valid_ar = X_valid[:, ~cond_mask]
X_test_ar = X_test[:, ~cond_mask]
X_train_cond = X_train[:, cond_mask]
X_valid_cond = X_valid[:, cond_mask]
X_test_cond = X_test[:, cond_mask]

In [8]:
# make supervised learning problem
X_train_ar = processing.series_to_supervised(X_train_ar, n_in=N_PAST, n_out=0, dropnan=True)
X_train_cond = processing.series_to_supervised(X_train_cond, n_in=1, n_out=0, dropnan=True)
y_train = processing.series_to_supervised(y_train, n_in=N_PAST, n_out=N_OUT, dropnan=True).iloc[:,-1]
X_valid_ar = processing.series_to_supervised(X_valid_ar, n_in=N_PAST, n_out=0, dropnan=True)
X_valid_cond = processing.series_to_supervised(X_valid_cond, n_in=1, n_out=0, dropnan=True)
y_valid = processing.series_to_supervised(y_valid, n_in=N_PAST, n_out=N_OUT, dropnan=True).iloc[:,-1]
X_test_ar = processing.series_to_supervised(X_test_ar, n_in=N_PAST, n_out=0, dropnan=True)
X_test_cond = processing.series_to_supervised(X_test_cond, n_in=1, n_out=0, dropnan=True)
y_test = processing.series_to_supervised(y_test, n_in=N_PAST, n_out=N_OUT, dropnan=True).iloc[:,-1]

# Align X with y
X_train_ar = X_train_ar[X_train_ar.index.isin(y_train.index)]
X_valid_ar = X_valid_ar[X_valid_ar.index.isin(y_valid.index)]
X_test_ar = X_test_ar[X_test_ar.index.isin(y_test.index)]
X_train_cond = X_train_cond[X_train_cond.index.isin(y_train.index)]
X_valid_cond = X_valid_cond[X_valid_cond.index.isin(y_valid.index)]
X_test_cond = X_test_cond[X_test_cond.index.isin(y_test.index)]

# Set to numpy arrays
X_train = X_train_ar.values
X_train_cond = X_train_cond.values
y_train = y_train.values
X_valid = X_valid_ar.values
X_valid_cond = X_valid_cond.values
y_valid = y_valid.values
X_test = X_test_ar.values
X_test_cond = X_test_cond.values
y_test = y_test.values

In [9]:
from cond_rnn import ConditionalRNN
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error, mean_absolute_error

def fit_cond_lstm(X, y, X_val, y_val, X_extra, X_val_extra, batch_size, nb_epochs, neurons, n_past):
    X = X.reshape(-1, n_past, int(X.shape[1] / n_past))
    X_val = X_val.reshape(-1, n_past, int(X_val.shape[1] / n_past))
    
    print(X.shape, X_extra.shape)

    # define model
    model = Sequential(layers=[
        ConditionalRNN(neurons, cell='LSTM'),
        Dense(units=1, activation='linear') # regression problem.
    ])

    model.compile(loss='mse', optimizer='adam')
    # fit model
    history = model.fit(
        [X, X_extra], y, epochs=nb_epochs, batch_size=batch_size,
        verbose=1, validation_data=([X_val, X_val_extra], y_val),
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=10, mode='auto')
        ])
    return model, history

In [10]:
lstm_model, history = fit_cond_lstm(
    X_train, y_train, 
    X_valid, y_valid, 
    X_train_cond, X_valid_cond,
    BATCH_SIZE, N_EPOCHS, N_NEURONS, 
    n_past=N_PAST)

(35003, 48, 5) (35003, 4)
Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
69/69 [==============================] - ETA: 0s - loss: 0.0446WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'tuple'> input: (<tf.Tensor 'IteratorGetNext:0' shape=(None, 48, 5) dtype=float32>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 4) dtype=float32>)
Consider rewriting this model with the Functional API.
69/69 [==============================] - 18s 231ms/step - loss: 0.0444 - val_loss: 0.0334
Epoch 2/100
69/69 [==============================] - 14s 208ms/step - loss: 0.0305 - val_loss: 0.0328
Epoch 3/100
69/69 [==============================] - 14s 205ms/step - loss: 0.0303 - val_loss: 0.0322
Epoch 4/100
69/69 [==============================] - 14s 200ms/step - loss: 0.0288 - val_loss: 0.0311
Epoch 5/100
69/69 [==============================] - 14s 204ms/step - loss: 0.0

In [11]:
def make_predictions(model, X, X_extra, y, scaler, n_past):
    # shape input data
    X = X.reshape(-1, n_past, int(X.shape[1]/n_past))
    # make predictions
    y_predict = model.predict([X, X_extra])
    # scale outputs back
    y_true = scaler.inverse_transform(y.reshape(-1,1)).flatten()
    y_predict = scaler.inverse_transform(y_predict.reshape(-1,1)).flatten()
    # evaluate predictions
    print("MAE: {} RMSE: {}".format(
        round(mean_absolute_error(y_true, y_predict), 3),
        round(np.sqrt(mean_squared_error(y_true, y_predict)),3)))
    return y_predict, y_true

In [12]:
# evaluate predictions
y_predict, y_true = make_predictions(lstm_model, X_valid, X_valid_cond, y_valid, scaler_output, N_PAST)

Consider rewriting this model with the Functional API.
MAE: 264.505 RMSE: 365.795


In [13]:
# With time features (LSTM) Horizon  1: MAE: 34.281 RMSE: 52.417
# With time features (LSTM) Horizon 12: MAE: 262.161 RMSE: 355.49
# With all features:  Horizon 12: MAE: 264.505 RMSE: 365.795

# Without time features horizon 1: MAE: 35.562 RMSE: 54.093 sMAPE: 0.361
# Without time features Horizon 12: MAE: 261.492 RMSE: 358.674 sMAPE: 0.877

# ---- (Lets try univariate + info):
# Horizon 1: MAE: 35.653 RMSE: 54.978 sMAPE: 0.394
# Horizon 12: MAE: 271.414 RMSE: 368.373